In [3]:
import numpy as np
import soundfile as sf
from UX_discrete_cepstrum import discrete_cepstrum_basic, discrete_cepstrum_reg

In [4]:
x, fs = sf.read('la.wav')

In [9]:
w = np.hanning(2049)
N = 2048
H = 512
t = -50
c = np.zeros(10)

M = w.size # analysis window size
N2 = N // 2 + 1 # size of positive spectrum
soundlength = x.size # length of input sound array
hM = int((M - 1) / 2) # half analysis window size
pin = hM # initialize sound pointer in middle of analysis window
pend = soundlength - hM # last sample to start a frame
fftbuffer = np.zeros(N, np.complex) # initialize buffer for FFT
yw = np.zeros(M) # initialize output sound frame
y = np.zeros(soundlength) # initialize output array
w /= w.sum() # normalize analysis window
sw = np.hanning(M) # synthesis window
sw /= sw.sum()
while pin < pend:
    #-----analysis-----
    xw = x[pin - hM : pin + hM + 1] * w # window the input sound
    fftbuffer[:] = 0 # reset buffer
    fftbuffer[0 : int((M + 1) / 2)] = xw[int((M - 1) / 2) : M] # zero-phase window in fftbuffer
    fftbuffer[N - (M - 1) // 2 : N] = xw[0 : (M - 1) // 2]
    X = np.fft.fft(fftbuffer)
    mX = 20 * np.log10(np.finfo(float).eps + np.abs(X[0:N2]))
    pX = np.unwrap(np.angle(X[0:N2])) # unwrapped phase spect. of positive freq.
    ploc = np.argwhere((mX[1 : N2 - 1] > t).astype(int) * (mX[1:N2-1] > mX[2:N2]).astype(int) * (mX[1:N2-1] > mX[0:N2-2]).astype(int)) # peaks
    pmag = mX[ploc] # magnitude of peaks
    pphase = pX[ploc] # phase of peaks
    # c = discrete_cepstrum_basic(ploc, pmag, 20)
    c = discrete_cepstrum_reg(ploc, pmag, 10, 0.9)
    #-----synthesis-----
    Y = np.zeros(N, np.complex) # initialize output spectrum
    Y[ploc] = 10 ** (pmag / 20) * np.exp(1j * pphase) # generate positive freq.
    Y[N - 1 - ploc] = 10 ** (pmag / 20) * np.exp(-1j * pphase) # generate negative freq.
    # generate neg.freq.
    fftbuffer = np.real(np.fft.ifft(Y)) # inverse FFT
    yw[0 : (M - 1) // 2] = fftbuffer[N - (M - 1) // 2 : N] # undo zero-phase window
    yw[(M - 1) // 2 : M] = fftbuffer[0 : (M + 1) // 2]
    y[pin - hM : pin + hM + 1] = y[pin - hM : pin + hM + 1] + H * N * sw * yw[0:M] # overlap-add
    pin = pin + H # advance sound pointer

/var/folders/lt/md23qtts5nxbd89hcfsdkqdh0000gn/T/ipykernel_41146/557765051.py:13: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fftbuffer = np.zeros(N, np.complex) # initialize buffer for FFT


LinAlgError: Singular matrix